# Glider Speed polars analysis notebook

In [1]:
import numpy as np
import math
from numpy.polynomial import Polynomial

print(np.__version__)

# https://www.geeksforgeeks.org/python-implementation-of-polynomial-regression/?ref=lbp
# old vs new API https://numpy.org/doc/1.21/reference/routines.polynomials.html

1.21.1


In [2]:
import glider.polar as gp

FILE_TO_LOAD = './glider-polars-db.json'

GLIDER_NAME = 'LAK19-18m'

polars_db = gp.PolarsDB.get_instance(FILE_TO_LOAD)

aGliderPolar = polars_db.fromNameAndSource(GLIDER_NAME,'Manual')

polars_ref_data = []
polars_ref_data.append(polars_db.fromNameAndSource(GLIDER_NAME,'SeeYou-1'))
polars_ref_data.append(polars_db.fromNameAndSource(GLIDER_NAME,'SeeYou-2'))
polars_ref_data.append(polars_db.fromNameAndSource(GLIDER_NAME,'iGlide'))

In [3]:
def polar_to_string(p):
	polar_str = '{}x<sup>2</sup>'.format(round(p.coeffs[0],4))
	polar_str += (' + ' if (p.coeffs[1] >= 0 ) else ' ')
	polar_str += '{}x'.format(round(p.coeffs[1],4))
	polar_str += (' + ' if (p.coeffs[2] >= 0 ) else ' ')
	polar_str += '{}'.format(round(p.coeffs[2],4))
	return polar_str

In [4]:
# class for extracting information from the plotly.graph_object Figure()
class TraceInfo:
	def __init__(self, figure):
		self.fig = figure
		self.traces = self.__traces()
		self.number = self.__len__()
		self.names = self.__trace_names()
		self.annotations = self.__annotations_names()

	def __getitem__(self, index):
		return self.traces[index]

	def __len__(self):
		return len(self.traces)

	def __traces(self):
		return self.fig['data']

	def __trace_names(self):
		return [trace['name'] for trace in self.traces]

	def __annotations_names(self):
		return [annotation['name'] for annotation in self.fig['layout']['annotations']]
	
	def trace_index(self, name_to_find, startWith = False):
		idx = 0
		for name in self.names:
			if (not startWith ) and (name == name_to_find):
				return idx
			elif (startWith and name.startswith(name_to_find)):
				return idx
			idx +=1 
		return None
	
	def annotation_index(self, name_to_find):
		idx = 0
		for annotation in self.annotations:
			if annotation.startswith(name_to_find):
				return idx
			idx +=1 
		return None


In [5]:
def update_polar(new_weight, figure):
	# Update the polar for the new weight
	aGliderPolar.update_wing_loading(new_weight)
	new_x_polar = np.linspace(gp.POLAR_CURVE_START_KM,gp.POLAR_CURVE_END_KM,gp.POLAR_CURVE_NBR_SAMPLE)
	new_y_polar = aGliderPolar.curve(new_x_polar)
	y_ref_polar = aGliderPolar.init_curve(new_x_polar)

	# Min sink rate update
	tg_x =  np.arange(0,250)
	tgh_y, msr_x, msr_y = aGliderPolar.tangent_horizontal(tg_x)

	# Max glide ratio update
	tgao_y, mgr_x, mgr_y = aGliderPolar.tangent_at_origin(tg_x)

	# Update existing traces
	t = TraceInfo(figure)

	polar_trace_idx = t.trace_index('Adjust',True)
	figure.data[polar_trace_idx].x = new_x_polar
	figure.data[polar_trace_idx].y = new_y_polar

	tgt_hz_trace_idx = t.trace_index('Horizontal tangent')
	figure.data[tgt_hz_trace_idx].x = tg_x
	figure.data[tgt_hz_trace_idx].y = tgh_y

	tgt_ao_trace_idx = t.trace_index('Tangent at (0,0)')
	figure.data[tgt_ao_trace_idx].x = tg_x
	figure.data[tgt_ao_trace_idx].y = tgao_y

	msr_marker_idx = t.trace_index('Min sink rate')
	figure.data[msr_marker_idx].x = msr_x
	figure.data[msr_marker_idx].y = msr_y

	mgr_marker_idx = t.trace_index('Max glide ratio')
	figure.data[mgr_marker_idx].x = mgr_x
	figure.data[mgr_marker_idx].y = mgr_y

	# Update annotation min sink rate
	min_sink_rate = aGliderPolar.get_min_sink_rate()
	text_annotation = '<b>Min sink rate</b><br> speed: {}km/h<br> Vz: {}m/s<br>L/D: {}'.format(round(min_sink_rate[0],1),round(min_sink_rate[1],2), round(min_sink_rate[2],1) )
	msr_a_idx = t.annotation_index('min-sink-rate')
	figure['layout']['annotations'][msr_a_idx].x = min_sink_rate[0]
	figure['layout']['annotations'][msr_a_idx].y = min_sink_rate[1]
	figure['layout']['annotations'][msr_a_idx].text = text_annotation

	# Update annotation max glide ratio
	max_glide_ratio = aGliderPolar.get_max_glide_ratio()
	text_annotation = '<b>Max glide ratio</b><br> speed: {}km/h<br> Vz: {}m/s<br>L/D: {}'.format(round(max_glide_ratio[0],1),round(max_glide_ratio[1],2), round(max_glide_ratio[2],1) )
	mgr_a_idx = t.annotation_index('max-glide-ratio')
	figure['layout']['annotations'][mgr_a_idx].x = max_glide_ratio[0]
	figure['layout']['annotations'][mgr_a_idx].y = max_glide_ratio[1]
	figure['layout']['annotations'][mgr_a_idx].text = text_annotation

	# Update annotation polynomial of the polar curve
	text_annotation = 'Polynomial equation for the polar:<br> {}'.format( polar_to_string(aGliderPolar.polynomial))
	polynomial_a_idx = t.annotation_index('polynomial')
	figure['layout']['annotations'][polynomial_a_idx].text = text_annotation

	# return adjusted_polar
	return None


In [6]:
# Display the graph

# https://plotly.com/python/plotly-fundamentals/
# https://plotly.com/python-api-reference/
# https://ipywidgets.readthedocs.io/en/stable/

import plotly.graph_objects as go
import plotly.express as px
from ipywidgets import widgets, Layout, Label
import ipywidgets as iw

traces=[]
colors = px.colors.qualitative.Plotly

# add reference polar curves
x_polar = np.linspace(gp.POLAR_CURVE_START_KM,gp.POLAR_CURVE_END_KM,gp.POLAR_CURVE_NBR_SAMPLE)
for aPolar in polars_ref_data:
		y_polar = aPolar.curve(x_polar)
		trace = go.Scatter( x=x_polar, y=y_polar, mode='lines', 
			name = '<b>{} - {}</b><br><i>(wing loading {}kg/m2, method {})</i><br>'.format(aPolar.name,aPolar.source, aPolar.wing_loading, aPolar.method()),
			hovertemplate='<extra></extra>Speed: %{x:.0f}km/h<br>Sink rate: %{y:.2f}m/s', hoverinfo='x+y', line = dict(dash='dash'))
		traces.append(trace)

# add traces for the model curve, the one we want to modify parameters (weight or wing loading) and interpolate new polar curve
if aGliderPolar is not None:
	y_polar = aGliderPolar.curve(x_polar)
	traces.append(go.Scatter( x=x_polar, y=y_polar, mode='lines', 
		name = '<b>{}</b><br><i>{}</i>'.format(aGliderPolar.name,aGliderPolar.source),
		hovertemplate='<extra></extra>Speed: %{x:.0f}km/h<br>Sink rate: %{y:.2f}m/s', hoverinfo='x+y', ))
	
	if isinstance(aGliderPolar, gp.PolarGlider3Points):
		traces.append(go.Scatter( x=aGliderPolar.speed, y=aGliderPolar.sink_rate, mode='markers', name='p1, p2, p3 points', marker_symbol = 'diamond',marker_size=10,
			text=['Point #1', 'Point #2', 'Point #3'], hovertemplate='<extra></extra><b>%{text}</b><br>Speed: %{x}km/h<br>Sink rate: %{y}m/s', hoverinfo='x+y+text'))
	
	# trace tanget at origin (0,0) and horizontal one
	tg_x =  np.arange(0,250)
	tgh_y, x_int, y_int = aGliderPolar.tangent_horizontal(tg_x)
	tg_trace = go.Scatter( x=tg_x, y=tgh_y, mode='lines', name = 'Horizontal tangent', line = dict(dash='dash') )
	msr_marker = go.Scatter( x=x_int, y=y_int, mode='markers', name = 'Min sink rate', marker_symbol = 'x-dot',marker=dict(color=colors[2], size=10), 
		hovertemplate='<extra></extra>Speed: %{x:.1f}km/h<br>Sink rate: %{y:.2f}m/s', hoverinfo='x+y' )
	traces.append(tg_trace)
	traces.append(msr_marker)

	tgao_y, x_int, y_int = aGliderPolar.tangent_at_origin(tg_x)
	tg_trace = go.Scatter( x=tg_x, y=tgao_y, mode='lines', name = 'Tangent at (0,0)', line = dict(dash='dash') )
	msr_marker = go.Scatter( x=x_int, y=y_int, mode='markers', name = 'Max glide ratio', marker_symbol = 'circle',marker=dict(color=colors[3], size=10),
			hovertemplate='<extra></extra>Speed: %{x:.1f}km/h<br>Sink rate: %{y:.2f}m/s', hoverinfo='x+y' )
	traces.append(tg_trace)
	traces.append(msr_marker)

	# then trace the curve that will be able to adjust base on wing-loading
	traces.append(go.Scatter( x=x_polar, y=y_polar, mode='lines+markers', 
		name = 'Adjusted (wing loading: {} kg/m2)'.format(aGliderPolar.wing_loading ),
		hovertemplate='<extra></extra>Speed: %{x:.0f}km/h<br>Sink rate: %{y:.2f}m/s', hoverinfo='x+y', ))

layout = go.Layout(
		title_text='<b>{} - Speed polars analysis</b><br><span class="font-size: smaller;">(wing area: {}m2)</span>'.format(GLIDER_NAME, aGliderPolar.wing_area),
		height=800,
		yaxis=dict(range=[-4, 1]),
		xaxis=dict(range=[0, 250]),
		template='ggplot2',
	)

fig = go.Figure(data=traces, layout=layout)

fig.update_yaxes(zeroline=True, zerolinewidth=1, zerolinecolor='black',  dtick=0.5)
fig.update_yaxes(title_text='Vz (m/s)', title_font=dict(size=14, family='Courier', color='crimson'))

fig.update_xaxes(zeroline=True, zerolinewidth=1, zerolinecolor='black')
fig.update_xaxes(title_text='Vitesse ({})'.format(( 'm/s' if gp.CONVERT_TO_MS else 'km/h')),
	 title_font=dict(size=14, family='Courier', color='crimson'))

if aGliderPolar is not None:
	text_annotation = 'Polynomial equation for the polar:<br> {}'.format( polar_to_string(aGliderPolar.polynomial))
	fig.add_annotation(
			name = 'polynomial',
			x=50, y=0.4, 
			text=text_annotation,
			bgcolor="white",
			borderpad=8,
			bordercolor='gray',
			borderwidth=4,
			showarrow=False,
		)
	
	# Min sink rate annotation
	min_sink_rate = aGliderPolar.get_min_sink_rate()
	text_annotation = '<b>Min sink rate</b><br> speed: {}km/h<br> Vz: {}m/s<br>L/D: {}'.format(round(min_sink_rate[0],1),round(min_sink_rate[1],2), round(min_sink_rate[2],1) )
	# text_annotation = '<b>Min sink rate</b>: speed: {}km/h, Vz: {}m/s, L/D: {}'.format(round(min_sink_rate[0],1),round(min_sink_rate[1],2), round(min_sink_rate[2],1) )
	fig.add_annotation(
			name = 'min-sink-rate',
			# x=xaxis_unit(50), y=0.2, 
			x=min_sink_rate[0], y=min_sink_rate[1], 
			text=text_annotation,
			bgcolor="white", borderpad=8, bordercolor='gray', borderwidth=2,
			showarrow=True,
			yshift=-5,
			ax= -10, ay= 80,
			arrowhead=2,  arrowsize=1, arrowwidth=2,
		)

	# Max glide ratio annotation
	max_gilde_ratio = aGliderPolar.get_max_glide_ratio()
	text_annotation = '<b>Max glide ratio</b><br> speed: {}km/h<br> Vz: {}m/s<br>L/D: {}'.format(round(max_gilde_ratio[0],1),round(max_gilde_ratio[1],2), round(max_gilde_ratio[2],1) )
	fig.add_annotation(
			name = 'max-glide-ratio',
			x=max_gilde_ratio[0], y=max_gilde_ratio[1], 
			text=text_annotation,
			bgcolor="white", borderpad=8, bordercolor='gray', borderwidth=2,
			showarrow=True,
			yshift=5,xshift=5,
			ax= 100, ay= -50,
			arrowhead=2,  arrowsize=1, arrowwidth=2,
		)

fig.update_layout(xaxis_range=[0,max(traces[0].x)*1.2])

# use FigureWiget insead of fig.show() to add controls
# fig.show()
f2 = go.FigureWidget(fig)

weights = widgets.FloatSlider(
	value= aGliderPolar.wing_loading,
	min=29,
	max=52,
	step=0.1,
	description= 'wing_loading',
	disabled=False,
	continuous_update=True,
	# orientation='vertical',
	readout=False,
	readout_format='.1f',
)
weight_text = Label(value=f'{weights.value:.1f} kg/m2', description='', layout=Layout(width='80px'))

container = widgets.HBox(children=[weights,weight_text,], layout=Layout(margin='0px 0px 100px 100px'))

def on_value_change(change):
	weight_text.value = f'{weights.value:.1f} kg/m2'

	with f2.batch_update():
		update_polar(weights.value,f2)
		
		t = 'Adjusted (wing loading: {} kg/m2)'.format(weights.value)
		f2.data[-1].name = t

weights.observe(on_value_change, names="value")
widgets.VBox([f2, container])


    'data': [{'hoverinfo': 'x+y',
              'hovertemplate': '<extra></extra…

/usr/local/lib/python3.9/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning:

The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.

